In [1]:
import os 
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    ! git clone --depth 1 https://github.com/tensorflow/models

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


In [3]:
utils_ops.tf = tf.compat.v1

tf.gfile = tf.io.gfile

In [4]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
                    fname = model_name,
                    origin = base_url + model_file,
                    untar = True)
    
    model_dir = pathlib.Path(model_dir)/"saved_model"
    
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']
    
    return model


In [5]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,use_display_name=True)


In [6]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]
    
    output_dict = model(input_tensor)
    
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy()
                      for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections
    
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
    if 'detection_masks' in output_dict:
        
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                output_dict['detection_masks'],output_dict['detection_boxes'],
                image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                                          tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
        
    return output_dict


In [26]:
from datetime import datetime

def truck_detection(image,classes,scores,boxes):
    for i in range(10):        
        if (classes[i]==8 and scores[i]>0.5):
            
            h,w = image.shape[0:2]

            ymin,xmin,ymax,xmax = boxes[i]
            
            now = datetime.now() 
            dt_string = now.strftime('%d_%m_%Y_%H_%M_%S')
            
            centre=(int(((xmin+xmax)/2)*w),int(((ymin+ymax)/2)*h))
            cv2.circle(image,centre,10,(0,0,255),-1) # radius = 10, line thickness = -1 field circle

            file_name = os.path.join('D:/ANACONDA/Project-9/source_material/models-master/research/object_detection/Custom_images',dt_string+'.jpg')
            cv2.imwrite(file_name,image)


In [27]:
def show_inference(model, image_path):
    image_np = image_path
    image_np = cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks = output_dict.get('detection_masks_reframed',None),
        use_normalized_coordinates=True,
        line_thickness=2)
    
    image_np = cv2.cvtColor(image_np,cv2.COLOR_RGB2BGR)
    truck_detection(image_np,output_dict['detection_classes'],output_dict['detection_scores'],
                 output_dict['detection_boxes'])
    return image_np

In [46]:
import cv2

video = cv2.VideoCapture(r'D:/ANACONDA/Project-9/source_material/models-master/research/object_detection/Custom_images/Pedestrian_video.mp4')
i = 0 # frame counter
while(True):
    ret = video.grab()   # grab frame
    i = i+1             # increment counter
    if i % 3 == 0:       # display only one third of the frames, you can change this parameters according to your needs
        ret, img = video.read()           # decode frame
        img = show_inference(detection_model,img)
        k=cv2.waitKey(1)
        cv2.imshow('Live',img)
        if(k==27):
            break
cv2.destroyAllWindows()


In [ ]:
import pandas as pd 
df = pd.DataFrame(category_index)
category_index = df.to_dict('series')
print(category_index)